Link: https://www.tensorflow.org/get_started/get_started

In [2]:
import tensorflow as tf

In [3]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [4]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [6]:
#Basic arithmetic
node3 = tf.multiply(node1,node2)
node4 = tf.exp(node3)
print("node3: ", node3)
print("node4: ", node4)
print("sess.run(node3): ",sess.run(node3))
print("sess.run(node4): ",sess.run(node4))

node3:  Tensor("Mul_1:0", shape=(), dtype=float32)
node4:  Tensor("Exp:0", shape=(), dtype=float32)
sess.run(node3):  12.0
sess.run(node4):  162755.0


In [14]:
#Placeholders
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
power_node = a ** b
sess.run(power_node, {a: 3, b:3})

27.0

**Placeholders** are parameters that you pass to the model. **Variables** are internal trainable parameters within the computational graph.

In [15]:
#Variables
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b
#Initialization
init = tf.global_variables_initializer()
sess.run(init)
#We can evaluate linear_model for several values of x simultaneously
sess.run(linear_model, {x:range(1,5)})

array([ 0.        ,  0.30000001,  0.60000002,  0.90000004], dtype=float32)

In [22]:
#Loss function
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:range(1,5), y:list(reversed(range(-3,1)))}))

23.66


In [24]:
#Assigning values to the variables
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


In [30]:
#Optimizing the parameters
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

sess.run([W, b])

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]

In [37]:
#Using tf.contrib.learn
import tensorflow as tf
import numpy as np

features = [tf.contrib.layers.real_valued_column("x")]

estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x_train}, y_train,
                                              batch_size=4,
                                              num_epochs=1000)
eval_input_fn = tf.contrib.learn.io.numpy_input_fn(
    {"x":x_eval}, y_eval, batch_size=4, num_epochs=1000)

estimator.fit(input_fn = input_fn, steps=1000)

train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_evaluation_master': '', '_save_checkpoints_secs': 600, '_model_dir': 'C:\\Users\\Tobias\\AppData\\Local\\Temp\\tmpq8l592ae', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000C803240>, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_save_checkpoints_steps': None, '_num_worker_replicas': 0, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_environment': 'local', '_save_summary_steps': 100, '_task_id': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_type': None, '_session_config': None, '_master': ''}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no lo

In [39]:
#Custom models with tf.contrib.learn
import numpy as np
import tensorflow as tf
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_train}, y_train, 4, num_epochs=1000)

# train
estimator.fit(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did. 
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_evaluation_master': '', '_save_checkpoints_secs': 600, '_model_dir': 'C:\\Users\\Tobias\\AppData\\Local\\Temp\\tmp56cgkj85', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000CAA1198>, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_save_checkpoints_steps': None, '_num_worker_replicas': 0, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_environment': 'local', '_save_summary_steps': 100, '_task_id': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_type': None, '_session_config': None, '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Tobias\AppData\Local\Temp\tmp56cgkj85\model.ckpt.
INFO:tensorflow:loss = 105.010479952, step = 1
INFO:tensorflow:global_step/sec: 1302.09
INFO:tensorflow:loss = 0.131063829075, step = 101 (0.062 sec)
INFO:tensorflo